In [ ]:
from dataprep.clean import clean_lat_long
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
import pyproj

- https://dados.gov.pt/en/datasets/distritos-de-portugal/
- https://epsg.io/4326
- https://epsg.io/20790
- https://pyproj4.github.io/pyproj/stable/api/proj.html#pyproj-proj
- https://epsg.io/3395
- https://geopandas.org/en/stable/docs/user_guide/projections.html#re-projecting
- https://snirh.apambiente.pt/index.php?idMain=1&idItem=1.3&salbufeirasimbolo=22L/01A
- https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html#dissolve-example
- https://gis.stackexchange.com/a/280314
- https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
- http://www.igeoe.pt/coordenadas/trans.aspx
- https://www.u-consult.pt/UConsultToolApps/Views/Conversor.html
- https://docs.dataprep.ai/api_reference/dataprep.clean.html#module-dataprep.clean.clean_lat_long
- https://pt.wikipedia.org/wiki/Barragem_da_Vigia
- https://geopandas.org/en/stable/docs/user_guide/projections.html#setting-a-projection
- https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.simplify.html

In [ ]:
PORTUGAL_MAP: str = "./raw/distritos/distritos.shp"

# Vigia
COORDENADA_M: int = 245862
COORDENADA_P: int = 174662

IN_EPSG: int = 20790
OUT_EPSG: int = 4326

In [ ]:
gdf = geopandas.read_file(PORTUGAL_MAP).query("TYPE_1 == 'Distrito'")

In [ ]:
gdf.head()

In [ ]:
gdf.plot(figsize=(15, 10))

In [ ]:
gdf.crs

In [ ]:
country_gdf = gdf.dissolve(by="NAME_0")

In [ ]:
country_gdf.shape

In [ ]:
country_gdf.crs

In [ ]:
country_gdf.plot(figsize=(15, 10))

In [ ]:
transformer = pyproj.Transformer.from_crs(f"epsg:{IN_EPSG}", f"epsg:{OUT_EPSG}")
coords = transformer.transform(COORDENADA_M, COORDENADA_P)
coords

In [ ]:
# https://docs.mapbox.com/help/glossary/lat-lon/

# Ranges:
# -90 to 90 for latitude
# -180 to 180 for longitude

In [ ]:
dataprep_df = pd.DataFrame({"coords": [coords]})

clean_lat_long(dataprep_df, "coords", split=True, output_format="dms")

In [ ]:
dataprep_df = pd.DataFrame({"coords": ["38° 32′ 17.152″ N, 7° 36′ 25.463″ W"]})

clean_lat_long(dataprep_df, "coords", split=True, output_format="dd")

In [ ]:
df = pd.DataFrame(
    {"Albufeira": ["Vigia"], "Latitude": [coords[0]], "Longitude": [coords[1]]}
)
df

In [ ]:
point_gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude)
)
point_gdf

In [ ]:
# point_gdf.crs

point_gdf = point_gdf.set_crs(epsg=OUT_EPSG)
point_gdf.crs

In [ ]:
point_gdf

In [ ]:
ax = country_gdf.plot(color="black")
# ax = country_gdf.plot(figsize=(15, 10), color="black")
# ax = country_gdf.plot(color="white", edgecolor="black", linewidth=0.5)
# ax = country_gdf.simplify(0.05).plot()

point_gdf.plot(ax=ax, color="red", linewidth=0)

plt.axis("off")
plt.margins(0)

plt.savefig(
    f"./minimap_vigia.svg",
    format="svg",
    transparent=True,
    bbox_inches="tight",
    pad_inches=0,
    backend="cairo",
)

plt.show()

---